In [1]:
import databaseconfig as cfg
import pandas as pd

import json

In [2]:
from sqlalchemy import create_engine
address='postgresql://'+cfg.postgres['user']+':'+cfg.postgres['pw']+'@'+cfg.postgres['host']+':5432/'+cfg.postgres['db']
engine = create_engine(address)
conn = engine.connect()

In [3]:
graffiti_df = pd.read_sql("SELECT * FROM GraffitiData LIMIT 10", conn)
graffiti_df

,id,creation_date,status,completion_date,type_of_service_request,surface_type,graffiti_location,zip,ward,community_area,latitude,longitude
0,1,2017-08-07,Completed,2017-08-07,Graffiti Removal,Metal - Painted,Door,60629,18,66,41.763870,-87.686746
1,2,2017-08-05,Completed,2017-08-07,Graffiti Removal,Wood - Painted,Front,60625,47,4,41.961625,-87.689770
2,3,2017-08-07,Completed,2017-08-07,Graffiti Removal,Metal - Painted,Dumpster,60608,28,28,41.864972,-87.666236
3,4,2017-08-07,Completed,2017-08-07,Graffiti Removal,Metal - Painted,Dumpster,60618,35,21,41.945151,-87.707931
4,5,2017-08-05,Completed,2017-08-07,Graffiti Removal,Aluminum Siding,Alley,60639,31,20,41.927055,-87.739237
5,6,2017-08-05,Completed,2017-08-07,Graffiti Removal,Metal - Painted,Door,60647,32,22,41.915129,-87.685262
6,7,2017-08-05,Completed,2017-08-07,Graffiti Removal,Other/Unknown Surface,Alley,60638,13,56,41.788384,-87.793054
7,8,2017-08-07,Completed,2017-08-07,Graffiti Removal,Brick - Unpainted,Side,60632,14,56,41.800183,-87.742809
8,9,2017-08-07,Completed,2017-08-07,Graffiti Removal,Metal - Painted,Garage,60609,12,59,41.827458,-87.675057
9,10,2017-08-07,Completed,2017-08-07,Graffiti Removal,Glass,Front,60647,1,22,41.915491,-87.704047


In [4]:
#graffiti_ward_count_df = pd.read_sql("SELECT ward,surface_type,graffiti_location,COUNT(id) FROM GraffitiData GROUP BY ward,surface_type,graffiti_location", conn)
graffiti_com_area_count_df = pd.read_sql("SELECT community_area,COUNT(id) FROM GraffitiData GROUP BY community_area", conn)

In [5]:
graffiti_com_area_count_df

,community_area,count
0,0.0,353
1,1.0,13050
2,2.0,17477
3,3.0,14407
4,4.0,22517
...,...,...
74,74.0,878
75,75.0,479
76,76.0,735
77,77.0,11818


In [6]:
data_dict = {}
for com_area in range(0,77):
    query_string = "SELECT surface_type,COUNT(id) FROM GraffitiData WHERE community_area="+str(com_area)+" GROUP BY surface_type ORDER BY count desc;"
    query_string = str(query_string)
    com_surface_df = pd.read_sql(query_string, conn)
    com_suf_df_len = len(com_surface_df)
    top_ten_df = com_surface_df.iloc[0:9]
    if 'Other/Unknown Surface' in top_ten_df['surface_type'].to_list():
        top_ten_df = com_surface_df.iloc[0:10]
        add_me = top_ten_df[top_ten_df['surface_type']=='Other/Unknown Surface']['count']
        top_ten_df = top_ten_df[top_ten_df['surface_type']!='Other/Unknown Surface']
        other_sum = sum(com_surface_df.iloc[11:com_suf_df_len]['count'].to_list())+add_me
    else:
        other_sum = sum(com_surface_df.iloc[10:com_suf_df_len]['count'].to_list())
    top_ten_df = top_ten_df.append({'surface_type':'Others/Unknown Surface','count':other_sum},ignore_index=True)
    data_dict[str(com_area+1)]={}
    data_dict[str(com_area+1)]['surface']={}
    temp_list = top_ten_df['surface_type'].apply(str).to_list()
    temp_list.reverse()
    data_dict[str(com_area+1)]['surface']['types']=temp_list
    temp_list = top_ten_df['count'].apply(str).to_list()
    temp_list.reverse()
    data_dict[str(com_area+1)]['surface']['counts']=temp_list

In [7]:
com_area = 0
query_string = "SELECT graffiti_location,COUNT(id) FROM GraffitiData WHERE community_area="+str(com_area)+" GROUP BY graffiti_location ORDER BY count"
ward_loc_df = pd.read_sql(query_string, conn)
ward_loc_df_len = len(ward_loc_df)
ward_loc_df

,graffiti_location,count
0,Phone Box,1
1,Overpass,1
2,Garbage Cart,2
3,Mail Box,4
4,Express Way Job,5
5,Door,5
6,Newspaper Box,5
7,Fence,6
8,None,7
9,Dumpster,7


In [8]:
for com_area in range(0,77):
    query_string = "SELECT graffiti_location,COUNT(id) FROM GraffitiData WHERE community_area="+str(com_area)+" GROUP BY graffiti_location ORDER BY count desc"
    com_loc_df = pd.read_sql(query_string, conn)
    com_loc_df_len = len(com_loc_df)
    top_ten_df = com_loc_df.iloc[0:9]
    if 'None' in top_ten_df['graffiti_location'].to_list():
        top_ten_df = com_loc_df.iloc[0:10]
        add_me = top_ten_df[top_ten_df['graffiti_location']=='None']['count']
        top_ten_df = top_ten_df[top_ten_df['graffiti_location']!='None']
        other_sum = sum(com_loc_df.iloc[11:com_loc_df_len]['count'].to_list())+add_me
    else:
        other_sum = sum(com_loc_df.iloc[10:com_loc_df_len]['count'].to_list())
    top_ten_df = top_ten_df.append({'graffiti_location':'Others/None','count':other_sum},ignore_index=True)
    data_dict[str(com_area+1)]['COUNT'] = str(sum(top_ten_df['count']))
    data_dict[str(com_area+1)]['location']={}
    temp_list = top_ten_df['graffiti_location'].apply(str).to_list()
    temp_list.reverse()
    data_dict[str(com_area+1)]['location']['types']=temp_list
    temp_list = top_ten_df['count'].apply(str).to_list()
    temp_list.reverse()
    data_dict[str(com_area+1)]['location']['counts']=temp_list

In [9]:
census_df = pd.read_csv('Per_Capita_Income.csv')
census_df

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
0,1.0,Rogers Park,7.7,23.6,8.7,18.2,27.5,23939,39.0
1,2.0,West Ridge,7.8,17.2,8.8,20.8,38.5,23040,46.0
2,3.0,Uptown,3.8,24.0,8.9,11.8,22.2,35787,20.0
3,4.0,Lincoln Square,3.4,10.9,8.2,13.4,25.5,37524,17.0
4,5.0,North Center,0.3,7.5,5.2,4.5,26.2,57123,6.0
...,...,...,...,...,...,...,...,...,...
73,74.0,Mount Greenwood,1.0,3.4,8.7,4.3,36.8,34381,16.0
74,75.0,Morgan Park,0.8,13.2,15.0,10.8,40.3,27149,30.0
75,76.0,O'Hare,3.6,15.4,7.1,10.9,30.3,25828,24.0
76,77.0,Edgewater,4.1,18.2,9.2,9.7,23.8,33385,19.0


In [10]:
census_cols = census_df.columns
census_cols = census_cols[1:len(census_cols)]
for com_area in range(1,78):
    for col in census_cols:
        data_dict[str(com_area)][col] = str(census_df[col].iloc[com_area-1])
#    print(str(com_area)+census_df['COMMUNITY AREA NAME'].iloc[com_area-1])

In [11]:
census_pop_df = pd.read_excel("CCASF12010CMAP.xlsx", skiprows=[0])
census_pop_df

,Geog,GeogKey,Total Population,"Not Hispanic or Latino, White alone","Not Hispanic or Latino, Black or African American alone","Not Hispanic or Latino, American Indian and Alaska Native alone","Not Hispanic or Latino, Asian alone","Not Hispanic or Latino, Native Hawaiian and Other Pacific Islander alone","Not Hispanic or Latino, Some Other Race alone","Not Hispanic or Latino, Two or More Races",...,Median Age,Total Households,Average Household Size,Total Housing Units,Occupied Housing Units,Vacant Housing Units,Occupied Housing Units.1,Owned with a mortgage or a loan,Owned free and clear,Renter occupied
0,Rogers Park,1,54991,21618,14461,142,3523,39,202,1573,...,32.404941,24292,2.13,27564,24292,3272,24292,6128,1009,17155
1,West Ridge,2,71942,30706,8015,189,16184,3,225,1919,...,35.062344,24755,2.85,27466,24755,2711,24755,8829,3367,12559
2,Uptown,3,56362,29098,11275,191,6414,10,130,1235,...,35.464521,29719,1.79,32541,29719,2822,29719,8298,1230,20191
3,Lincoln Square,4,39493,24916,1488,72,4399,11,128,917,...,33.450642,18194,2.14,19906,18194,1712,18194,5482,1406,11306
4,North Center,5,31867,24634,738,55,1449,16,51,594,...,32.478928,14561,2.18,16197,14561,1636,14561,5805,1147,7609
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72,Washington Heights,73,26493,121,25793,36,12,4,18,251,...,41.225061,9482,2.77,10365,9482,883,9482,4626,1921,2935
73,Mount Greenwood,74,19093,16413,987,34,126,3,6,142,...,36.110942,6990,2.63,7374,6990,384,6990,4440,1555,995
74,Morgan Park,75,22544,6473,15039,11,92,8,24,278,...,40.467193,8187,2.70,8958,8187,771,8187,4400,1534,2253
75,O'Hare,76,12756,9845,403,15,1058,2,24,197,...,36.626579,6127,2.08,6677,6127,550,6127,1843,881,3403


In [12]:
census_pop_df[census_pop_df['Geog']=='West Lawn']

,Geog,GeogKey,Total Population,"Not Hispanic or Latino, White alone","Not Hispanic or Latino, Black or African American alone","Not Hispanic or Latino, American Indian and Alaska Native alone","Not Hispanic or Latino, Asian alone","Not Hispanic or Latino, Native Hawaiian and Other Pacific Islander alone","Not Hispanic or Latino, Some Other Race alone","Not Hispanic or Latino, Two or More Races",...,Median Age,Total Households,Average Household Size,Total Housing Units,Occupied Housing Units,Vacant Housing Units,Occupied Housing Units.1,Owned with a mortgage or a loan,Owned free and clear,Renter occupied
64,West Lawn,65,33355,4924,1390,28,210,3,18,113,...,30.000717,8766,3.8,9362,8766,596,8766,5135,1446,2185


In [13]:
# Save numer of complaints per 1000 residents
for com_area in range(0,77):
    data_dict[str(com_area+1)]['COUNT PER CAPITA'] = str(round(1.0*int(data_dict[str(com_area+1)]['COUNT'])/census_pop_df.iloc[com_area]['Total Population']*1000,3))

# Getting quartile data for box plots

In [22]:
# Query used to get sample of elapsed times
# temp_df = pd.read_sql("""
#      SELECT creation_date,completion_date,DATE_PART('day',completion_date::timestamp - creation_date::timestamp) AS time_elapsed
#      FROM GraffitiData WHERE (creation_date>'2000-01-01') AND (creation_date is not null) AND (completion_date is not null)
#      LIMIT 100
# """,conn)
com_quartile_df =pd.read_sql("""
WITH myquery AS (
     SELECT community_area,DATE_PART('day',completion_date::timestamp - creation_date::timestamp) AS time_elapsed
     FROM GraffitiData WHERE (creation_date>'2000-01-01') AND (creation_date is not null) AND (completion_date is not null) AND (community_area is not null)
)
SELECT community_area,
percentile_cont(0.0) within group (order by time_elapsed) as min, 
percentile_cont(0.25) within group (order by time_elapsed) as q1, 
percentile_cont(0.5) within group (order by time_elapsed) as q2,
percentile_cont(0.75) within group (order by time_elapsed) as q3,
percentile_cont(1.0) within group (order by time_elapsed) as max
FROM myquery GROUP BY community_area
""", conn)
com_quartile_df

,community_area,min,q1,q2,q3,max
0,0,0.0,0.0,0.0,0.0,129.0
1,1,0.0,0.0,1.0,4.0,326.0
2,2,0.0,0.0,1.0,3.0,137.0
3,3,0.0,0.0,0.0,2.0,153.0
4,4,0.0,0.0,1.0,2.0,241.0
...,...,...,...,...,...,...
73,73,0.0,0.0,0.0,1.0,54.0
74,74,0.0,0.0,0.0,0.0,43.0
75,75,0.0,0.0,0.0,1.0,51.0
76,76,0.0,0.0,0.0,1.0,101.0


In [23]:
surf_quartile_df =pd.read_sql("""
WITH myquery AS (
     SELECT surface_type,DATE_PART('day',completion_date::timestamp - creation_date::timestamp) AS time_elapsed
     FROM GraffitiData WHERE (creation_date>'2000-01-01') AND (creation_date is not null) AND (completion_date is not null) AND (surface_type is not null)
)
SELECT surface_type,
percentile_cont(0.0) within group (order by time_elapsed) as min, 
percentile_cont(0.25) within group (order by time_elapsed) as q1, 
percentile_cont(0.5) within group (order by time_elapsed) as q2,
percentile_cont(0.75) within group (order by time_elapsed) as q3,
percentile_cont(1.0) within group (order by time_elapsed) as max
FROM myquery GROUP BY surface_type
""", conn)
surf_quartile_df

,surface_type,min,q1,q2,q3,max
0,Aluminum Siding,0.0,1.0,1.0,4.0,130.0
1,Asphalt,0.0,1.0,1.0,3.0,105.0
2,Brick - Painted,0.0,0.0,0.0,1.0,2527.0
3,Brick - Unpainted,0.0,0.0,1.0,5.0,3286.0
4,"Cement (Sidewalk, Alley, Wall, Curb)",0.0,0.0,0.0,2.0,366.0
5,Glass,0.0,0.0,0.0,1.0,366.0
6,Limestone,0.0,1.0,1.0,3.0,2890.0
7,Marble/Granite,0.0,0.0,1.0,3.0,104.0
8,Metal,0.0,1.0,1.0,5.0,3064.0
9,Metal - Painted,0.0,0.0,0.0,1.0,2348.0


In [24]:
loc_quartile_df =pd.read_sql("""
WITH myquery AS (
     SELECT graffiti_location,DATE_PART('day',completion_date::timestamp - creation_date::timestamp) AS time_elapsed
     FROM GraffitiData WHERE (creation_date>'2000-01-01') AND (creation_date is not null) AND (completion_date is not null) AND (graffiti_location is not null)
)
SELECT graffiti_location,
percentile_cont(0.0) within group (order by time_elapsed) as min, 
percentile_cont(0.25) within group (order by time_elapsed) as q1, 
percentile_cont(0.5) within group (order by time_elapsed) as q2,
percentile_cont(0.75) within group (order by time_elapsed) as q3,
percentile_cont(1.0) within group (order by time_elapsed) as max
FROM myquery GROUP BY graffiti_location
""", conn)
loc_quartile_df

,graffiti_location,min,q1,q2,q3,max
0,Alley,0.0,1.0,1.0,4.0,522.0
1,Bench,0.0,0.0,0.0,1.0,366.0
2,Bridge,0.0,0.0,0.0,1.0,6.0
3,Concrete Pillar,0.0,0.0,0.0,1.0,13.0
4,Door,0.0,0.0,0.0,1.0,330.0
5,Door Way,0.0,0.0,0.0,0.0,3.0
6,Dumpster,0.0,0.0,0.0,0.0,366.0
7,Express Way Job,0.0,0.0,1.0,3.0,171.0
8,Express Way Job - DSS will NOT remove,0.0,0.0,0.0,0.0,0.0
9,Fence,0.0,0.0,0.0,1.0,366.0


In [27]:
box_plots_dict = {}
box_plots_dict['comunity areas']={}
for com_area in range(0,77):
        box_plots_dict['comunity areas'][str(com_area+1)]={}
        box_plots_dict['comunity areas'][str(com_area+1)]['COMMUNITY AREA NAME']=data_dict[str(com_area+1)]['COMMUNITY AREA NAME']
        five_num_summary = com_quartile_df.loc[com_area].apply(str).to_list()[1:6]
        box_plots_dict['comunity areas'][str(com_area+1)]['com_quartiles']=[five_num_summary[0],five_num_summary[1],five_num_summary[1],
                                                                            five_num_summary[2],five_num_summary[3],five_num_summary[3],five_num_summary[4]]

In [30]:
box_plots_dict['surfaces']={}
index = 0
for type in surf_quartile_df['surface_type'].to_list():
    five_num_summary = surf_quartile_df.loc[index].apply(str).to_list()[1:6]
    box_plots_dict['surfaces'][type]=[five_num_summary[0],five_num_summary[1],five_num_summary[1],
                                    five_num_summary[2],five_num_summary[3],five_num_summary[3],five_num_summary[4]]
    index+=1

In [31]:
box_plots_dict['locations']={}
index = 0
for type in loc_quartile_df['graffiti_location'].to_list():
    five_num_summary = loc_quartile_df.loc[index].apply(str).to_list()[1:6]
    box_plots_dict['locations'][type]=[five_num_summary[0],five_num_summary[1],five_num_summary[1],
                                    five_num_summary[2],five_num_summary[3],five_num_summary[3],five_num_summary[4]]
    index+=1

In [32]:
census_df.sort_values('PER CAPITA INCOME ')

,Community Area Number,COMMUNITY AREA NAME,PERCENT OF HOUSING CROWDED,PERCENT HOUSEHOLDS BELOW POVERTY,PERCENT AGED 16+ UNEMPLOYED,PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA,PERCENT AGED UNDER 18 OR OVER 64,PER CAPITA INCOME,HARDSHIP INDEX
53,54.0,Riverdale,5.8,56.5,34.6,27.5,51.5,8201,98.0
29,30.0,South Lawndale,15.2,30.7,15.8,54.8,33.8,10402,96.0
36,37.0,Fuller Park,3.2,51.2,33.9,26.6,44.9,10432,97.0
25,26.0,West Garfield Park,9.4,41.7,25.8,24.5,43.6,10934,92.0
66,67.0,West Englewood,4.8,34.4,35.9,26.3,40.7,11317,89.0
...,...,...,...,...,...,...,...,...,...
32,33.0,Near South Side,1.3,13.8,4.9,7.4,21.8,59077,7.0
5,6.0,Lake View,1.1,11.4,4.7,2.6,17.0,60058,5.0
31,32.0,Loop,1.5,14.7,5.7,3.1,13.5,65526,3.0
6,7.0,Lincoln Park,0.8,12.3,5.1,3.6,21.5,71551,2.0


In [33]:
box_plots_dict['PER CAPITA INCOME ORDER'] = census_df.sort_values('PER CAPITA INCOME ')['Community Area Number'].dropna().apply(int).apply(str).to_list()

In [35]:
# Dump dictionaries to file
with open("data_dict.json","w") as fp:
    json.dump(data_dict, fp)
with open("box_plots_dict.json","w") as fp:
    json.dump(box_plots_dict, fp)

In [19]:
data_dict["1"]

{'surface': {'types': ['Others/Unknown Surface',
   'Wood - Painted',
   'None',
   'Metal - Unpainted',
   'Metal',
   'Other / Unknown Surface',
   'Cement (Sidewalk, Alley, Wall, Curb)',
   'Brick - Unpainted',
   'Brick - Painted',
   'Metal - Painted'],
  'counts': ['11', '6', '6', '8', '10', '23', '40', '50', '55', '140']},
 'COUNT': '346',
 'location': {'types': ['Others/None',
   'None',
   'Rear',
   'Alley',
   'Garage',
   'Sign',
   'Traffic Control Box',
   'Side',
   'Pole',
   'Front'],
  'counts': ['29', '7', '8', '10', '11', '11', '23', '45', '88', '114']},
 'COMMUNITY AREA NAME': 'Rogers Park',
 'PERCENT OF HOUSING CROWDED': '7.7',
 'PERCENT HOUSEHOLDS BELOW POVERTY': '23.6',
 'PERCENT AGED 16+ UNEMPLOYED': '8.7',
 'PERCENT AGED 25+ WITHOUT HIGH SCHOOL DIPLOMA': '18.2',
 'PERCENT AGED UNDER 18 OR OVER 64': '27.5',
 'PER CAPITA INCOME ': '23939',
 'HARDSHIP INDEX': '39.0',
 'COUNT PER CAPITA': '6.292',
 'box_plot_data': ['0.0', '0.0', '0.0', '0.0', '0.0', '0.0', '129.0

In [20]:
data_dict["1"]['COMMUNITY AREA NAME']

'Rogers Park'